<h1 align="center"><font color='royalblue'> ADA - Project ADACONDA</font></h1> 

**Students:**
- ABI FADEL Zad
- ADEYE Abiola
- BRUNO Etienne
- FERCHIOU Sami

## IMPORTS AND SETTINGS

In [ ]:
# Imports all required libraries
import sqlite3
import plotly.express as px
import dask.dataframe as dd
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import pearsonr
from dask.distributed import Client
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
from utils.constants import BigMarketsSubset, SmallMarketsSubset, Correspondence, BigMarkets, SmallMarkets, MediumMarkets
from data.API_KEYS import chart_studio_key, chart_studio_username
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from math import log
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls
import plotly.express as px
import logging
import os

# Sets up logging to Plotly
username = chart_studio_username
api_key = chart_studio_key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

# Default settings for matplotlib
sns.set(rc={"figure.figsize": (8, 5)})
sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

colors_plt = px.colors.qualitative.Plotly

# Default colors for our market sizes
color_discrete_map = {
    "small": colors_plt[1],
    "big": colors_plt[2],
    "medium": colors_plt[0],
    "SMALL MARKET": colors_plt[1],
    "BIG MARKET": colors_plt[2],
    "MEDIUM MARKET": colors_plt[0],
}

In [ ]:
# Dask Client with number of workers equal to number of cores
n_workers = os.cpu_count()
client = Client(n_workers=10, silence_logs=logging.ERROR)
client

## DATA PREPARATION

<div class="alert alert-block alert-info">
Our first step is to prepare the dataframe on which we will conduct our analysis. We then load our data in parquet format and filter the categories of videos that are the most insightful for us. This step permits us to focus on the videos that are the most interesting for our research, remove noise from irrelevant categories and make our queries faster. <br>
Therefore, we filter our videos to only keep categories Sports, People & Blogs and Entertainment.

In [ ]:
video_metadata_df = dd.read_parquet("./data/video_metadata/parquet/")
video_metadata_df["upload_date"] = dd.to_datetime(video_metadata_df.upload_date)

In [ ]:
video_metadata_df_sports = video_metadata_df[video_metadata_df["categories"].isin(["Sports", "People & Blogs", "Entertainment"])]
video_metadata_df_sports["tags"] = video_metadata_df_sports["tags"].str.lower()
video_metadata_df_sports["tags"] = video_metadata_df_sports["tags"].apply(lambda x: f",{x},", meta=("tags", "object"))

In [ ]:
date_obj = video_metadata_df_sports.upload_date.dt
group_by = [date_obj.year, date_obj.month, video_metadata_df.categories]
video_metadata_df_sports_grouped = video_metadata_df_sports[["upload_date"]].groupby(by=group_by).count().persist()

In [ ]:
required_tags = ["nba", "basketball"]

# Verifying that each video contains the required tags
nba_basketball_df = video_metadata_df_sports[video_metadata_df_sports["tags"].str.contains(f',{",|,".join(required_tags)},')].persist()
nba_basketball_df["year_month"] = dd.to_datetime(
    nba_basketball_df.upload_date.dt.year.astype("str") + "-" + nba_basketball_df.upload_date.dt.month.astype("str")
)
nba_basketball_df.head(2)

In [ ]:
# Print dictionary of the big market teams
print("BigMarketsSubset")
for key, value in BigMarketsSubset.items():
    print(f"{key} : {value}")

print("\nSmallMarketsSubset")
# Print dictionary of the small market teams
for key, value in SmallMarketsSubset.items():
    print(f"{key} : {value}")

<div class="alert alert-block alert-info">
We groupby the videos according to their respective team, sum the number of views for each team on a monthly basis, label each team with its market size and finally calculate its view ratio compared to all NBA/Basketball views. 

In [ ]:
nba_basketball_views_per_year_df = nba_basketball_df[["year_month", "view_count"]].groupby("year_month").sum().persist().compute()
nba_basketball_views_per_year_df.rename(columns={"view_count": "total_nba_views"}, inplace=True)

dd_dict = {}
# Getting the total amount of views per month for every big market and small market team
for key, value in {**BigMarketsSubset, **SmallMarketsSubset}.items():
    print(key, value)
    team_df = nba_basketball_df[nba_basketball_df["tags"].str.contains(f',{",|,".join(value)},')]
    team_df = team_df[["year_month", "view_count"]].groupby("year_month").sum()
    team_df = team_df.reset_index()
    team_df["team"] = key
    team_df["market_size"] = "small" if key in SmallMarketsSubset else "big"
    team_df = team_df.persist().compute()
    dd_dict[key] = team_df

In [ ]:
all_team_results = pd.concat([v for k, v in dd_dict.items()])
all_team_results = all_team_results.merge(nba_basketball_views_per_year_df, on="year_month")
all_team_results["total_nba_view_percentage"] = all_team_results["view_count"] / all_team_results["total_nba_views"]
all_team_results["season"] = all_team_results["year_month"].apply(lambda x: f"{x.year - 1}-{x.year}" if x.month < 10 else f"{x.year}-{x.year + 1}")

# Saving the results to a csv file
all_team_results.to_csv("./data/all_team_results.csv", index=False)

## DATA VISUALIZATION FOR THE INTRODUCTION

### GENERATE VISUALIZATION FOR COMPARISON OF MAJOR SPORTS LEAGUES IN THE USA

In [ ]:
major_league_comparison = pd.read_csv("data/major_league_comparison.csv")
major_league_comparison

In [ ]:
major_league_comparison.plot(
    x="League",
    y=["Viewers (in millions)", "US TV rev. (in billions of $)"],
    kind="bar",
    rot=0,
    title="US major leagues viewership and revenue comparison",
)

# Display a legend for the chart
plt.legend()

# Save the chart to a file with a resolution of 300 dpi
plt.savefig("imgs/major_league_viewership_comparison.png", dpi=300)

### GENERATE VISUALIZATION OF EVOLUTION OF NBA VIDEOS

In [ ]:
df = nba_basketball_views_per_year_df.copy()

# Set 'year_month' column as index, sort by index in ascending order, and reset index
df.sort_index(inplace=True, ascending=True)
df.reset_index(inplace=True)

# Plot 'total_nba_views' against 'year_month' for rows with 'year_month' <= '2018-01-01'
df[df.year_month <= "2018-01-01"].plot(x="year_month", y="total_nba_views", legend=None)

# Customize plot
plt.xticks(rotation=45)
plt.xlabel("Date (monthly frequency)")
plt.ylabel("Aggregate views (in billions)")
plt.title('Viewership evolution of videos containing the keyword "NBA"', pad=15)
plt.tight_layout()

# Save plot as image file
plt.savefig("imgs/viewership_evolution_nba.jpeg", dpi=400)

## BIG MARKET VS SMALL MARKET TEAMS

### DATA VISUALISATION FOR THE MARKET SIZE COMPARISON

In [ ]:
def to_real_market_size(elem):
    if "M" in elem:
        return float(elem[:-1])
    if "K" in elem:
        return float(elem[:-1]) / 1000
    else:
        return np.nan


# Read the csv file into a pandas DataFrame
market_size_df = pd.read_csv("./data/market_sizes.csv")
market_size_df["TV MARKET SIZE"] = market_size_df["TV MARKET SIZE"].apply(to_real_market_size)
market_size_df["METRO POPULATION"] = market_size_df["METRO POPULATION"].apply(lambda x: x / (10**6))
market_size_df["MARKET TYPE"] = market_size_df["TV MARKET SIZE"].apply(
    lambda x: "BIG MARKET" if x > 2 else "SMALL MARKET" if x < 1.5 else "MEDIUM MARKET"
)
market_size_df.head()

In [ ]:
market_size_df

In [ ]:
# Set figure size and create subplot
fig = px.scatter(
    market_size_df,
    x="TV MARKET SIZE",
    y="METRO POPULATION",
    custom_data=["TEAM"],
    color="MARKET TYPE",
    color_discrete_map=color_discrete_map,
    size=market_size_df["METRO POPULATION"],
    log_x=True,
    log_y=False,
    width=1000,
    height=700,
)

# Add vertical lines
fig.add_shape(type="line", x0=2, x1=2, y0=0, y1=1, name="1.5M", yref="paper", line=dict(color="red", width=2))
fig.add_shape(type="line", x0=1.5, x1=1.5, y0=0, y1=1, yref="paper", line=dict(color="red", width=2))


# Add title and axis labels
fig.update_layout(
    xaxis_title="TV Market Size (in millions)",
    xaxis_title_font=dict(size=14),
    yaxis_title="Metro Population (in millions)",
    yaxis_title_font=dict(size=14),
)

# Add hover labels for team names
fig.update_traces(hovertemplate="<b>%{customdata[0]}</b>")
fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=12,
    )
)
fig.update_layout(yaxis_range=[0, 15])
fig.update_layout(xaxis_range=[0, 0.8])


# fig.update_layout(hovermode='x unified')
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=0.8))
fig.show()

py.plot(fig, filename="market_size_comparison", auto_open=False)

### STUDY OF THE DISTRIBUTION OF VIDEOS VIEWS

In [ ]:
# Eliminate limit years to avoid undiserable results when computing seasons
df = all_team_results.copy()
df = df[df.year_month.dt.year != 2006]
df = df[df.season != "2019-2020"]

In [ ]:
small_vs_big = df.groupby(["season", "market_size"]).sum().drop(["total_nba_views", "view_count"], axis=1)
small_vs_big.reset_index(inplace=True)
display(small_vs_big)

big_market_percentage_df = small_vs_big[small_vs_big["market_size"] == "big"]
small_market_percentage_df = small_vs_big[small_vs_big["market_size"] == "small"]

In [ ]:
plt.fill_between(
    big_market_percentage_df["season"],
    big_market_percentage_df["total_nba_view_percentage"],
    small_market_percentage_df["total_nba_view_percentage"],
    color="peachpuff",
    alpha=0.3,
)
plt.plot(big_market_percentage_df["season"], big_market_percentage_df["total_nba_view_percentage"], color="firebrick", label="Big Markets")
plt.plot(small_market_percentage_df["season"], small_market_percentage_df["total_nba_view_percentage"], color="steelblue", label="Small Markets")
plt.xlabel("Year")
plt.ylabel("Percentage")
plt.title("Small vs Big Markets Percentage of all NBA video views")
plt.legend()
plt.tight_layout()

# Rotate and align the tick labels so they look better
plt.setp(plt.gca().get_xticklabels(), rotation=45, horizontalalignment="right")

# Save the chart to a file with a resolution of 300 dpi
plt.savefig("imgs/small_vs_big_market_percentge_comparison.png", dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
percentage_per_season = df.groupby("season").sum().drop(["total_nba_views"], axis=1)

views_per_team = df.groupby(["season", "team"]).agg({"total_nba_view_percentage": "sum", "market_size": "first", "view_count": "sum"})

# Divide each team's views by the total views for that season
views_per_team["subset_percentage"] = views_per_team["total_nba_view_percentage"].div(
    percentage_per_season["total_nba_view_percentage"], axis=0, level="season"
)
views_per_team.rename(columns={"view_count": "Views"}, inplace=True)
views_per_team.reset_index(inplace=True)
views_per_team

# Create a plotly pie chart, where the season is an additional dimension
fig = px.pie(
    views_per_team,
    values="subset_percentage",
    names="team",
    color="team",
    title="Percentage of all NBA video views by team",
    color_discrete_map={"big": "firebrick", "small": "steelblue"},
    labels={"subset_percentage": "Percentage of all NBA video views by team"},
    hover_data=["market_size"],
    width=800,
    height=800,
)
# center the title
fig.update_layout(title_x=0.5)
fig

In [ ]:
# Create a bar plot displaying the percentage of views for each team as a fraction of the total views for that season
# and color the bars by market size
fig = px.bar(
    views_per_team,
    x="team",
    y=views_per_team.subset_percentage * 100,
    color="market_size",
    color_discrete_map=color_discrete_map,
    barmode="stack",
    color_discrete_sequence=px.colors.qualitative.Pastel,
    animation_frame="season",
    animation_group="team",
    custom_data=["Views"],
)

fig.update_layout(
    # title="Distribution of views inside our subset by team",
    xaxis_title="Team",
    yaxis_title="Percentage",
    legend_title="Market Size",
    title_x=0.5,
    yaxis_range=[0, 55],
)

# Add hover labels for team names
fig.update_traces(hovertemplate="<b>Number of views: %{customdata[0]}</b>")
fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=12,
        font_family="Ubuntu",
    )
)

py.plot(fig, filename="distribution_of_views_animation", auto_open=False)
fig

### DIRECTION OF FREE AGENT MOVES IN THE NBA BETWEEN 2020 and 2021

In [ ]:
# Read csv files for the free agent moves for the 2021 and 2022 seasons
free_agent_moves_2021 = pd.read_csv("./data/free_agent_moves/2021_free_agent_moves.csv")
free_agent_moves_2022 = pd.read_csv("./data/free_agent_moves/2022_free_agent_moves.csv")

# Concatenate the two dataframes
free_agent_moves = pd.concat([free_agent_moves_2021, free_agent_moves_2022])

# Keep only the rows where the NEWS column starts with "Signs with"
free_agent_moves = free_agent_moves[free_agent_moves["NEWS"].str.startswith("Signs with") | False]

# Add from and to columns to see direction of trade
free_agent_moves["TO"] = free_agent_moves["NEWS"].str.split(" ").str[2]
free_agent_moves["FROM"] = ""

# Save the concatenate dataframe to a csv file
# free_agent_moves.to_csv("./data/free_agent_moves/free_agent_moves.csv", index=False)

In [ ]:
# Reload file after manually completing "TO" column
free_agent_moves = pd.read_csv("./data/free_agent_moves/free_agent_moves.csv")
free_agent_moves.drop(["EXP", "NEWS", "TYPE", "POS"], axis=1, inplace=True)

In [ ]:
def map_market_size(team):
    if team in BigMarkets:
        return 2
    elif team in MediumMarkets:
        return 1
    else:
        return 0


free_agent_moves["TO_MARKET"] = free_agent_moves["TO"].map(map_market_size)
free_agent_moves["FROM_MARKET"] = free_agent_moves["FROM"].map(map_market_size)
free_agent_moves

In [ ]:
df_from_large_to_small = free_agent_moves[(free_agent_moves["PPG"] > 0) & (free_agent_moves["TO_MARKET"] < free_agent_moves["FROM_MARKET"])]
display(df_from_large_to_small)

df_from_small_to_large = free_agent_moves[(free_agent_moves["PPG"] > 0) & (free_agent_moves["TO_MARKET"] > free_agent_moves["FROM_MARKET"])]
display(df_from_small_to_large)

df_eq_moves = free_agent_moves[(free_agent_moves["PPG"] > 0) & (free_agent_moves["TO_MARKET"] == free_agent_moves["FROM_MARKET"])]
display(df_eq_moves)

### STUDY ON THE CORRELATION BETWEEN VIEWS AND WINNING PERCENTAGE

In [ ]:
all_team_results = pd.read_csv("./data/all_team_results.csv", parse_dates=["year_month"])
all_team_results.head()

<div class="alert alert-block alert-info">
To compare our view ratio with the teams' results, we import from our database the monthly occupancy rate of the stadium of each team and the winning rate on the matches since the beginning of each season.

In [ ]:
# Connect to the database
con = sqlite3.connect("data/nba_api.sqlite")

# Read the data from the database
occupancy_rate_df = pd.read_sql_query(
    """
    select   home,
                avg(attendance) as avg_attendace,
                round(avg(attendance/season_high_attendance*100),2) as avg_occupancy_rate,
                strftime('%Y-%m',time) as year_month
    from game_data
    group by home, year_month
    """,
    con,
)
occupancy_rate_df = occupancy_rate_df.reset_index()
display(occupancy_rate_df.home.unique())  # Display query results

# Filter out teams that are not in the correspondence dictionary
occupancy_rate_df = occupancy_rate_df[occupancy_rate_df["home"].isin(Correspondence.keys())]

winning_rate_df = pd.read_sql_query(
    """
    select  home as team,
            strftime('%Y-%m', time) as year_month,
            last_value(curr_season_win_pct) over (partition by home, strftime('%Y-%m', time) order by time) as win_percentage
            from game_data
            group by home, year_month;
""",
    con,
)
winning_rate_df = winning_rate_df.reset_index()

# Filter out teams that are not in the correspondence dictionary
winning_rate_df = winning_rate_df[winning_rate_df["team"].isin(Correspondence.keys())]

winning_rate_df["year_month"] = pd.to_datetime(winning_rate_df["year_month"])
occupancy_rate_df["year_month"] = pd.to_datetime(occupancy_rate_df["year_month"])

winning_rate_df["team"] = winning_rate_df["team"].apply(lambda x: Correspondence[x])
occupancy_rate_df["team"] = occupancy_rate_df["home"].apply(lambda x: Correspondence[x])

winning_rate_df.head(10)

In [ ]:
# Add winning percentages and occupancy percentages for every team to all team results df
all_team_results = all_team_results.merge(winning_rate_df.drop(["index"], axis=1), on=["team", "year_month"], how="inner")
all_team_results = all_team_results.merge(occupancy_rate_df.drop(["index", "home"], axis=1), on=["team", "year_month"], how="inner")

# Save to csv since computation is expensive
all_team_results.to_csv("data/all_team_results_with_winning_percentages.csv")

all_team_results.head()

In [ ]:
# Add a new column that is the mean of view_counts per year_month
all_team_results["mean_view_count"] = all_team_results.groupby("year_month")["view_count"].transform("mean")
all_team_results["std_view_count"] = all_team_results.groupby("year_month")["view_count"].transform("std").fillna(1)

all_team_results["view_count_normalized"] = (all_team_results["view_count"] - all_team_results["mean_view_count"]) / all_team_results[
    "std_view_count"
]
all_team_results["year"] = all_team_results["year_month"].dt.year

In [ ]:
# compute correlation between view_count and win_percentage for both big and small market teams
grouped_views_and_wins_df = all_team_results.groupby("market_size").agg(
    {
        "view_count_normalized": lambda x: list(x),
        "win_percentage": lambda x: list(x),
    }
)

correlations = {}
for market_size in ["small", "big"]:
    df = grouped_views_and_wins_df[grouped_views_and_wins_df.index == market_size]

    correlations[market_size] = pearsonr(df["view_count_normalized"].iloc[0], df["win_percentage"].iloc[0])
correlations

## NBA TEAM MEDIA EXPOSURE ANALYSIS

### MEDIA COVERAGE OF EACH MARKET ON YOUTUBE

#### NUMBER OF VIDEOS PER TEAM

In [ ]:
dd_dict = {}
# Getting the total amount of views per month for every big market and small market team
for key, value in {**BigMarketsSubset, **SmallMarketsSubset}.items():
    print(key, value)
    team_df = nba_basketball_df[nba_basketball_df["tags"].str.contains(f',{",|,".join(value)},')]
    team_df = team_df[["year_month", "view_count"]].groupby("year_month").sum()
    team_df = team_df.reset_index()
    team_df["team"] = key
    team_df["market_size"] = "small" if key in SmallMarketsSubset else "big"
    team_df = team_df.persist().compute()
    dd_dict[key] = team_df

In [ ]:
dd_dict = {}
# Selecting the teams and their market size depending on the tags of the videos
for key, value in {**BigMarkets, **MediumMarkets, **SmallMarkets}.items():
    team_df = nba_basketball_df[nba_basketball_df["tags"].str.contains(f',{",|,".join(value)},')]
    team_df["team"] = key
    team_df["market_size"] = "small" if key in SmallMarkets else "medium" if key in MediumMarkets else "big"
    team_df = team_df.persist().compute()
    dd_dict[key] = team_df

all_team_df = pd.concat([v for k, v in dd_dict.items()])
all_team_df.head()

In [ ]:
# Computing the number of videos for each team
videos_per_team_df = all_team_df[["team", "market_size", "title"]].groupby(["team", "market_size"]).count()
videos_per_team_df.rename(columns={"title": "total_videos"}, inplace=True)

In [ ]:
fig = px.bar(
    videos_per_team_df.sort_values("total_videos").reset_index(),
    x="team",
    y="total_videos",
    labels={"team": "Team", "total_videos": "Total Number of Videos"},
    color="market_size",
    color_discrete_map=color_discrete_map,
    width=800,
)
fig.update_layout(title={"text": "Number of Videos per Team", "x": 0.5, "xanchor": "center"})
fig.update_layout(xaxis_categoryorder="total ascending")
fig.show()

py.plot(fig, filename="nb_videos_per_team", auto_open=False)

#### NUMBER OF VIDEOS PER MARKET

In [ ]:
# Computing the number of videos for each market
videos_per_market_df = all_team_df[["market_size", "title"]].groupby(["market_size"]).count()
videos_per_market_df.rename(columns={"title": "total_videos"}, inplace=True)

In [ ]:
fig = px.pie(
    videos_per_market_df.reset_index(),
    values="total_videos",
    names="market_size",
    labels={"market_size": "Market Size", "total_videos": "Total Number of Videos"},
    color="market_size",
    color_discrete_map=color_discrete_map,
    width=400,
)
fig.update_layout(title={"text": "Number of Videos per Market Size", "x": 0.5, "xanchor": "center"})
fig.update_traces(textinfo="value+percent")
fig.show()

py.plot(fig, filename="nb_videos_market_size", auto_open=False)

### NUMBER OF CHANNELS PER TEAM

In [ ]:
# Computing the ratio of videos related to a single team on each channel
channel_team_df = all_team_df[["title", "channel_id", "team"]].groupby(["channel_id", "team"]).count().reset_index()
count_by_channel = channel_team_df.groupby("channel_id").sum().reset_index()
count_by_channel.rename(columns={"title": "total_videos"}, inplace=True)
channel_team_df.rename(columns={"title": "team_videos"}, inplace=True)
ratio_per_team = pd.merge(channel_team_df, count_by_channel, on="channel_id")
ratio_per_team["ratio"] = ratio_per_team["team_videos"] / ratio_per_team["total_videos"]

# Only selecting channels that uploaded a significant number of videos
significant_channels_df = ratio_per_team[ratio_per_team.total_videos > 4]

# Select only fanbase channels - were at least 60% of videos are on a single team
fanbase_channels_df = significant_channels_df[significant_channels_df.ratio > 0.51]

# Computing the number of fanbase channels per team
fanbase_channels_df = fanbase_channels_df.groupby("team").count().sort_values("ratio")

# Adding the market size for each team
marketper_team_df = all_team_df[["team", "market_size"]].groupby(["team", "market_size"]).count().reset_index()
fanbase_channels_df = fanbase_channels_df.merge(marketper_team_df, on="team")

In [ ]:
# Visualizing the number of channels per team and keep the same color for each team
fig = px.bar(
    fanbase_channels_df,
    x="team",
    y="ratio",
    labels={"ratio": "Number of Fanbase channels", "team": "Team"},
    color="market_size",
    color_discrete_map=color_discrete_map,
    width=800,
)
# fig.update_layout(title={"text": "Number of FanBase Channels per Team", "x": 0.5, "xanchor": "center"})
fig.update_layout(xaxis_categoryorder="total ascending")
fig.show()

py.plot(fig, filename="fanbase_chaneneks_per_team", auto_open=False)

### THE BIG NAMES BIAS

In [ ]:
# Computing the fan engagement on each market size
engagement_df = all_team_df.groupby(["market_size"]).sum()
engagement_df["engagement"] = (engagement_df.like_count / engagement_df.view_count) * 1000

In [ ]:
# Visualizing the fans engagement on each market size
fig = px.bar(
    engagement_df.sort_values("engagement").reset_index(),
    x="market_size",
    y="engagement",
    width=600,
    labels={"market_size": "Market Size", "engagement": "Engagement ratio (Likes/Views x 1000)"},
    range_y=[7, 10],
    color="market_size",
    color_discrete_map=color_discrete_map,
)
# remove legend
fig.update_layout(showlegend=False)
# fig.update_layout(title={"text": "Engagement by Market Size", "x": 0.5, "xanchor": "center"})
fig.show()

py.plot(fig, filename="engagement_market_size_wt", auto_open=False)

### Impact of a star player on a team

In [ ]:
import sqlite3

con = sqlite3.connect("data/nba_api.sqlite")


# get df with winning percentage over last 15 games per team
df_win_pct = pd.read_sql(
    """
select abbreviation as team, round(last_15/15*100, 2) as win_pct, strftime('%Y-%m-01',time) as year_month 
from game_data gd 
join teams t on gd.home = t.full_name 
where last_15 is not null and year_month between '2006-01-01' and '2019-12-31'
group by abbreviation, time 
order by year_month desc
""",
    con,
)

In [ ]:
# obtain statistics on movement of views per team over time
df = pd.read_csv("data/28_nba_team_results.csv", parse_dates=["year_month"])

df.sort_values("year_month", inplace=True)
df["team_pct_increase_in_views"] = df["total_nba_views"].pct_change()
df_all_teams_pct_increase_in_views = (
    df[["year_month", "total_nba_views"]].groupby("year_month").agg(all_teams_pct_increase_in_views=("total_nba_views", sum)).reset_index()
)
df_all_teams_pct_increase_in_views["all_teams_pct_increase_in_views"] = df_all_teams_pct_increase_in_views[
    "all_teams_pct_increase_in_views"
].pct_change()
df = df.merge(df_all_teams_pct_increase_in_views)
df["team_vs_nba_pct_differential"] = df["team_pct_increase_in_views"] - df["all_teams_pct_increase_in_views"]
df = df[["year_month", "view_count", "team"]]

In [ ]:
team = "LAL"
important_signings = [("GSW", "OKC", "2016-07-04", "Kevin Durant"), ("CLE", "MIA", "2015-07-11", "LeBron James")]


import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

for (team_joined, team_left, date, player) in important_signings:
    team_df = df[df["team"] == team_joined]
    dmin, dmax = team_df.view_count.min(), team_df.view_count.max()
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add figure title
    fig.update_layout(title_text=f"Impact of {player} (left {team_left} for {team_joined})")

    # Set y-axes titles
    fig.update_yaxes(title_text="Team-related <b>youtube</b> views", secondary_y=False)

    team_df = df[df["team"] == team_joined]
    fig.add_trace(go.Scatter(x=team_df["year_month"], y=team_df["view_count"], mode="lines", name=team_joined))

    win_pct = df_win_pct[df_win_pct["team"] == team_joined]

    team_df = df[df["team"] == team_left]
    dmin, dmax = min(dmin, team_df.view_count.min()), max(dmax, team_df.view_count.max())
    fig.add_trace(go.Scatter(x=team_df["year_month"], y=team_df["view_count"], mode="lines", name=team_left))
    win_pct = df_win_pct[df_win_pct["team"] == team_left]

    fig.add_trace(go.Scatter(x=[date, date], y=[dmin, dmax], mode="lines", line=dict(color="green", width=2, dash="dash"), name=f"{player} signing"))
    fig.show()
    fig.write_image(f"data/event_study/{player}_{team_left}_{team_joined}.png")
    py.plot(fig, filename=f'event_study_{player.replace(" ",  "_")}_{team_left}_{team_joined}.png', auto_open=False)

In [ ]:
from data.API_KEYS import chart_studio_username, chart_studio_key
import chart_studio.plotly as py
import chart_studio

# Sets up logging to Plotly
username = chart_studio_username
api_key = chart_studio_key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)


import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

for (team_joined, team_left, date, player) in important_signings:
    team_df = df[df["team"] == team_joined]
    dmin, dmax = team_df.view_count.min(), team_df.view_count.max()
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    # Add figure title
    fig.update_layout(title_text=f"Impact of {player} (left {team_left} for {team_joined})")

    # Set y-axes titles
    fig.update_yaxes(title_text="Winning percentage over last 15 games", secondary_y=False)
    team_df = df[df["team"] == team_joined]

    win_pct = df_win_pct[df_win_pct["team"] == team_joined]
    fig.add_trace(
        go.Scatter(x=win_pct["year_month"], y=win_pct["win_pct"], mode="lines", line=dict(dash="dot"), name=f"{team_joined} win %"), secondary_y=True
    )

    team_df = df[df["team"] == team_left]
    dmin, dmax = min(dmin, team_df.view_count.min()), max(dmax, team_df.view_count.max())

    win_pct = df_win_pct[df_win_pct["team"] == team_left]
    fig.add_trace(
        go.Scatter(x=win_pct["year_month"], y=win_pct["win_pct"], mode="lines", line=dict(dash="dot"), name=f"{team_left} win %"), secondary_y=True
    )

    fig.add_trace(go.Scatter(x=[date, date], y=[dmin, dmax], mode="lines", line=dict(color="green", width=2, dash="dash"), name=f"{player} signing"))
    fig.show()
    fig.write_image(f"data/event_study_2/{player}_{team_left}_{team_joined}.png")
    py.plot(fig, filename=f'event_study_2_{player.replace(" ",  "_")}_{team_left}_{team_joined}.png', auto_open=False)